In [2]:
import pandas as pd
import numpy as np

In [3]:
train=pd.read_csv('creditcard 2.csv')

In [4]:
print (train.head())

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [10]:
print (train['Time'].max())

172792.0


In [11]:
print (train.shape)

(284807, 31)


In [12]:
print (train['Class'].value_counts())

0    284315
1       492
Name: Class, dtype: int64


In [13]:
#let's just start with a vanilla random forest model for this
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X = train.loc[:, train.columns != 'Class']
y = train['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

clf=RandomForestClassifier(n_estimators=100)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)



array([[85271,     8],
       [   40,   124]])

In [17]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.8780018756225978

In [16]:
print (np.bincount(y_pred))

[85311   132]


In [22]:
#let's just start with a vanilla random forest model for this
#Now we will be downsampling the 0s and see what happens

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X = train.loc[:, train.columns != 'Class']
y = train['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

#Here we will be downsampling the X_train and y_train
new_train = pd.concat([X_train,y_train],axis=1)
new_train_zeros=new_train[new_train.Class==0]
new_train_ones=new_train[new_train.Class==1]
new_train_zeros = new_train_zeros.sample(frac=0.3, replace=True, random_state=1)

train_final = pd.concat([new_train_zeros,new_train_ones],axis=0)
X_train = train_final.loc[:, train_final.columns != 'Class']
y_train = train_final['Class']

clf=RandomForestClassifier(n_estimators=100)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


array([[85279,    14],
       [   25,   125]])

In [21]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.9101391590926924

In [ ]:
#Now modifying the cost fuction of the SVM to account for class imbalance
